# Modelling most important features

## Model Selection

In [1]:
# Importing the libraries
import os
import pandas as pd

## Data Preprocessing

In [2]:
# Load and preprocess the train data 
from pipelines import preprocessing_pipeline

data_file = os.path.join('..', '..', '..', '..', 'data', 'raw', 'train.csv')
train_data = pd.read_csv(data_file, index_col=0, low_memory=False)
train_data = preprocessing_pipeline.fit_transform(train_data)
train_data.head()

,day_phase,bg-5:55,bg-5:50,bg-5:45,bg-5:30,bg-5:25,bg-5:20,bg-5:15,bg-5:10,bg-5:05,...,bg-0:50,bg-0:45,bg-0:30,bg-0:25,bg-0:20,bg-0:15,bg-0:10,bg-0:05,bg-0:00,bg+1:00
id,,,,,,,,,,,,,,,,,,,,,
p01_0,morning,9.6,9.6,9.6,9.7,9.533333,9.366667,9.2,9.033333,8.866667,...,17.433333,17.5,17.3,16.933333,16.566667,16.2,15.833333,15.466667,15.1,13.4
p01_1,morning,9.7,9.7,9.7,9.2,9.033333,8.866667,8.7,8.600000,8.500000,...,17.366667,17.3,16.2,15.833333,15.466667,15.1,14.866667,14.633333,14.4,12.8
p01_2,morning,9.2,9.2,9.2,8.7,8.600000,8.500000,8.4,8.300000,8.200000,...,16.566667,16.2,15.1,14.866667,14.633333,14.4,14.233333,14.066667,13.9,15.5
p01_3,morning,8.7,8.7,8.7,8.4,8.300000,8.200000,8.1,8.166667,8.233333,...,15.466667,15.1,14.4,14.233333,14.066667,13.9,13.866667,13.833333,13.8,14.8
p01_4,morning,8.4,8.4,8.4,8.1,8.166667,8.233333,8.3,8.733333,9.166667,...,14.633333,14.4,13.9,13.866667,13.833333,13.8,13.666667,13.533333,13.4,12.7


## Data Splitting

In [3]:
from sklearn.model_selection import train_test_split

X = train_data.drop(columns=['bg+1:00'])
y = train_data['bg+1:00']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Standardization

In [4]:
from pipelines import standardization_pipeline

X_train = standardization_pipeline.fit_transform(X_train)
X_test = standardization_pipeline.transform(X_test)

X_train.head()

,bg-5:55,bg-5:50,bg-5:45,bg-5:30,bg-5:25,bg-5:20,bg-5:15,bg-5:10,bg-5:05,bg-5:00,...,bg-0:20,bg-0:15,bg-0:10,bg-0:05,bg-0:00,day_phase_evening,day_phase_late_evening,day_phase_morning,day_phase_night,day_phase_noon
id,,,,,,,,,,,,,,,,,,,,,
p12_24040,-0.493061,-0.493210,-0.493555,-0.493993,-0.461122,-0.494014,-0.527083,-0.594125,-0.627184,-0.693575,...,-0.793842,-0.826333,-0.860824,-0.860843,-0.826242,0,0,0,0,1
p12_13267,0.071004,0.004753,-0.028365,-0.061718,-0.094928,-0.161159,-0.127596,-0.127526,-0.427238,-0.427020,...,-0.292711,-0.359165,-0.292777,-0.359590,-0.459099,0,0,1,0,0
p03_131,0.834151,0.801492,0.769103,0.869334,0.903782,0.937264,1.037576,1.072299,1.205655,1.238950,...,2.212943,2.243625,2.179898,1.979588,1.877267,0,0,0,0,0
p12_20861,0.203725,0.369925,0.569736,0.935838,0.903782,0.837407,0.871123,0.805671,0.705790,0.672520,...,-0.526572,-0.559380,-0.660337,-0.827426,-0.993125,0,0,0,0,0
p12_24627,-0.858044,-0.957974,-0.991972,-1.059275,-1.027058,-0.960012,-0.859989,-0.827425,-0.693832,-0.726895,...,-0.326120,-0.325796,-0.292777,-0.359590,-0.425722,0,0,0,0,0


## Model Selection with LazyPredict

In [5]:
from notebooks.helpers.LazyPredict import get_lazy_regressor

reg = get_lazy_regressor(exclude=['SVR'])
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
models

 97%|█████████▋| 37/38 [19:37<00:10, 10.53s/it] 

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.035179 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15227
[LightGBM] [Info] Number of data points in the train set: 141619, number of used features: 65
[LightGBM] [Info] Start training from score 8.276012


100%|██████████| 38/38 [19:42<00:00, 31.11s/it]


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
KNeighborsRegressor,0.81,0.82,1.29,5.19
ExtraTreesRegressor,0.70,0.70,1.65,231.13
BaggingRegressor,0.61,0.61,1.87,71.19
XGBRegressor,0.58,0.58,1.94,2.92
MLPRegressor,0.57,0.57,1.98,59.95
LGBMRegressor,0.56,0.56,1.99,4.57
HistGradientBoostingRegressor,0.56,0.56,1.99,9.33
GradientBoostingRegressor,0.53,0.53,2.05,119.05
LassoLarsCV,0.51,0.51,2.10,1.36


### We select the best models with RMSE < 2.0

In [6]:
models[models['RMSE'] < 2.0]

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
KNeighborsRegressor,0.81,0.82,1.29,5.19
ExtraTreesRegressor,0.70,0.70,1.65,231.13
BaggingRegressor,0.61,0.61,1.87,71.19
XGBRegressor,0.58,0.58,1.94,2.92
MLPRegressor,0.57,0.57,1.98,59.95
LGBMRegressor,0.56,0.56,1.99,4.57
HistGradientBoostingRegressor,0.56,0.56,1.99,9.33
